In [19]:
clear

In [25]:
function A = loadFunctionFromData(dataPath)
    disp(["🦍 Loading file: "  dataPath])
    load(dataPath)

    if isstruct(Problem.A)
        n = length(Problem.A.jc)-1;
        ii = repelems(1:n, [1:n; diff(Problem.A.jc)]);
        A = sparse(Problem.A.ir+1, ii, Problem.A.data);
    else
        A = Problem.A;
    end
endfunction

In [2]:
whos

Variables in the current scope:

   Attr Name        Size                     Bytes  Class
   ==== ====        ====                     =====  ===== 
        ans         1x5                          5  char

Total is 5 elements using 5 bytes



In [26]:
A = loadFunctionFromData('./data/cage15.mat');
columns(A)
rows(A)
clear
A = loadFunctionFromData('./data/vas_stokes_2M.mat');
columns(A)
rows(A)

🦍 Loading file: ./data/cage15.mat
ans =  5154859
ans =  5154859
🦍 Loading file: ./data/vas_stokes_2M.mat
ans =  2146677
ans =  2146677


In [17]:
whos
who

Variables in the current scope:

error: out of memory or dimension too large for Octave's index type
Variables in the current scope:

A    ans



In [53]:
function [x, e] = gmres( A, b, x, max_iterations, threshold)
  n = length(A);
  m = max_iterations;

  % use x as the initial vector
  r = b - A * x;

  b_norm = norm(b);
  error = norm(r) / b_norm;

  % initialize the 1D vectors
  sn = zeros(m, 1);
  cs = zeros(m, 1);
  %e1 = zeros(n, 1);
  e1 = zeros(m+1, 1);
  e1(1) = 1;
  e = [error];
  r_norm = norm(r);
  Q(:,1) = r / r_norm;
  beta = r_norm * e1;     %Note: this is not the beta scalar in section "The method" above but the beta scalar multiplied by e1
  for k = 1:m

    % run arnoldi
    [H(1:k+1, k) Q(:, k+1)] = arnoldi(A, Q, k);
    
    % eliminate the last element in H ith row and update the rotation matrix
%     [H(1:k+1, k) cs(k) sn(k)] = apply_givens_rotation(H(1:k+1,k), cs, sn, k);
    
%     % update the residual vector
%     beta(k + 1) = -sn(k) * beta(k);
%     beta(k)     = cs(k) * beta(k);
    error       = abs(beta(k + 1)) / b_norm;

    % save the error
    e = [e; error];

    if (error <= threshold)
      break;
    end
  end
  % if threshold is not reached, k = m at this point (and not m+1) 
  
  % calculate the result
  y = H(1:k, 1:k) \ beta(1:k);
  x = x + Q(:, 1:k) * y;
end

%----------------------------------------------------%
%                  Arnoldi Function                  %
%----------------------------------------------------%
function [h, q] = arnoldi(A, Q, k)
  q = A*Q(:,k);   % Krylov Vector
  for i = 1:k     % Modified Gram-Schmidt, keeping the Hessenberg matrix
    h(i) = q' * Q(:, i);
    q = q - h(i) * Q(:, i);
  end
  h(k + 1) = norm(q);
  q = q / h(k + 1);
end

%---------------------------------------------------------------------%
%                  Applying Givens Rotation to H col                  %
%---------------------------------------------------------------------%
function [h, cs_k, sn_k] = apply_givens_rotation(h, cs, sn, k)
  % apply for ith column
  for i = 1:k-1
    temp   =  cs(i) * h(i) + sn(i) * h(i + 1);
    h(i+1) = -sn(i) * h(i) + cs(i) * h(i + 1);
    h(i)   = temp;
  end

  % update the next sin cos values for rotation
  [cs_k sn_k] = givens_rotation(h(k), h(k + 1));

  % eliminate H(i + 1, i)
  h(k) = cs_k * h(k) + sn_k * h(k + 1);
  h(k + 1) = 0.0;
end

%%----Calculate the Givens rotation matrix----%%
function [cs, sn] = givens_rotation(v1, v2)
%  if (v1 == 0)
%    cs = 0;
%    sn = 1;
%  else
    t = sqrt(v1^2 + v2^2);
%    cs = abs(v1) / t;
%    sn = cs * v2 / v1;
    cs = v1 / t;  % see http://www.netlib.org/eispack/comqr.f
    sn = v2 / t;
%  end
end

In [2]:
rand("seed", 20);


In [3]:
dim = 20;
A = sprand(dim, dim, .5);
b = rand(dim, 1);
x0 = zeros(dim, 1);

In [54]:
 [ x, e ] = gmres(A, b, x0, 20, 1e-15);

In [55]:
x

x =

   0.1362378
   0.0748684
   0.2018766
   0.0379617
   0.1573230
   0.1127407
   0.0072511
   0.0530942
   0.0975751
   0.0317502
   0.2136133
   0.0022815
   0.1011048
   0.2721933
   0.1780554
   0.2397838
   0.0442396
   0.2030687
   0.0553625
   0.0237037



In [56]:
e

e =

   1
   0



In [57]:
real_res = A\b;

In [58]:
real_res - x

ans =

  -0.589621
  -0.075496
  -0.226904
   0.614768
  -0.526069
   0.264500
  -1.047154
  -0.490603
  -0.119053
   0.224256
  -0.183607
   0.698915
   0.244459
   0.629252
   0.522637
  -0.587955
   0.219638
  -0.159404
   0.042200
   0.030949



In [59]:
A

A =

Compressed Column Sparse (rows = 20, cols = 20, nnz = 200 [50%])

  (2, 1) ->  0.41217
  (3, 1) ->  0.039193
  (9, 1) ->  0.92475
  (12, 1) ->  0.31320
  (14, 1) ->  0.65118
  (16, 1) ->  0.61071
  (20, 1) ->  0.68544
  (2, 2) ->  0.78087
  (3, 2) ->  0.60937
  (4, 2) ->  0.10632
  (5, 2) ->  0.99955
  (8, 2) ->  0.85510
  (9, 2) ->  0.22975
  (10, 2) ->  0.085235
  (11, 2) ->  0.82960
  (13, 2) ->  0.75684
  (15, 2) ->  0.61077
  (16, 2) ->  0.78227
  (18, 2) ->  0.041342
  (19, 2) ->  0.79346
  (1, 3) ->  0.29006
  (2, 3) ->  0.10126
  (3, 3) ->  0.11989
  (5, 3) ->  0.56749
  (8, 3) ->  0.19417
  (10, 3) ->  0.44313
  (11, 3) ->  0.068074
  (17, 3) ->  0.92143
  (19, 3) ->  0.80411
  (20, 3) ->  0.43728
  (7, 4) ->  0.35427
  (10, 4) ->  0.10302
  (12, 4) ->  0.46300
  (14, 4) ->  0.68933
  (18, 4) ->  0.57330
  (1, 5) ->  0.85270
  (4, 5) ->  0.93643
  (7, 5) ->  0.99590
  (8, 5) ->  0.73317
  (9, 5) ->  0.44636
  (10, 5) ->  0.63989
  (11, 5) ->  0.97170
  (12, 5) ->  0.68865

In [63]:
ttt = load("../data/test.mst");

In [69]:
A = ttt.a;
b = ttt.b;
x0 = zeros(20, 1);

In [74]:
[ x, e ] = gmres(A, b', x0, 20, 1e-15);

In [75]:
x
e

x =

   0.0671786
   0.0778803
   0.1055276
   0.0761267
   0.1327323
   0.0094638
   0.0819292
   0.1204225
   0.1765256
   0.0793504
   0.0390035
   0.0223154
   0.2754362
   0.1220134
   0.1701944
   0.2691823
   0.0242330
   0.2567317
   0.1615334
   0.0482720

e =

   1
   0



In [77]:
real_res = A\b';
real_res- x 

ans =

  -2.022116
   0.641929
  -0.500320
  -1.491304
   0.239621
  -0.922369
  -0.097088
  -0.571888
   1.581124
  -1.269419
   1.954327
   1.599065
   0.198717
  -3.526679
  -2.267741
  -0.432758
   2.655671
  -3.327171
   0.981829
   3.186030



In [78]:
real_res

real_res =

  -1.954938
   0.719809
  -0.394793
  -1.415177
   0.372353
  -0.912906
  -0.015159
  -0.451465
   1.757650
  -1.190069
   1.993330
   1.621381
   0.474153
  -3.404666
  -2.097547
  -0.163576
   2.679904
  -3.070439
   1.143362
   3.234302



In [79]:
max(real_res- x )

ans =  3.1860
